論文  
https://arxiv.org/abs/2011.09055  

Dataset  
https://svip-lab.github.io/dataset/iPER_dataset.html  

GitHub  
https://github.com/iPERDance/iPERCore  
  
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/iPERCore_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ランタイムの設定
「ランタイム」→「ランタイムのタイプを変更」→「ハードウェアアクセラレータ」をGPUに変更

# 実行方法
「ランタイム」→「すべてのセルを実行」を選択

# 環境セットアップ

## GPUの確認

In [ ]:
!nvidia-smi

## ライブラリのインストール

In [ ]:
!pip install moviepy==0.2.3.5 imageio==2.4.1
!pip install yt-dlp

## ライブラリのインポート

In [ ]:
import os
import os.path as osp
import platform
import argparse
import time
import sys
import subprocess

from yt_dlp import YoutubeDL
from google.colab import files
from moviepy.editor import *
from moviepy.video.fx.resize import resize

## 環境変数設定

In [ ]:
!ls /usr/local/ | grep cuda

# CUDA_HOMEにcuda-11のパスを設定
os.environ["CUDA_HOME"] = "/usr/local/cuda-11"

!echo $CUDA_HOME

# iPERCoreのセットアップ

## GitHubからコードを取得

In [ ]:
!git clone https://github.com/iPERDance/iPERCore.git

# Commits on Oct 31, 2022
%cd /content/iPERCore
!git checkout 61f7e086316d7f64779be75d3a771da4e1e4d441

## ライブラリのインストール

In [ ]:
%cd /content/iPERCore/

# cuda11.2以上の場合cuda11.1の設定でセットアップ
!sed -i '158i \    version = version if version != 11.2 else 11.1' setup.py

!python setup.py develop

## 学習済みモデルのダウンロード

In [ ]:
# Download all checkpoints
#!wget -O assets/checkpoints.zip "https://1drv.ws/u/s!AjjUqiJZsj8whLkwQyrk3W9_H7MzNA?e=rRje0G"
!wget -O assets/checkpoints.zip "https://download.impersonator.org/iper_plus_plus_latest_checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/

!rm assets/checkpoints.zip

## サンプルデータのダウンロード

In [ ]:
%cd /content/iPERCore

# download samples
!wget -O assets/samples.zip  "http://101.32.75.151:10086/samples.zip"
!unzip -o assets/samples.zip -d  assets
!rm assets/samples.zip

# download checkpoints
!wget -O assets/checkpoints.zip "http://101.32.75.151:10086/checkpoints.zip"
!unzip -o assets/checkpoints.zip -d assets/

!rm assets/checkpoints.zip

# Motion Imitation

## configの詳細
  --gpu_ids（str）：gpu_ids、デフォルトは「0」です  
  --image_size（int）：画像サイズ。デフォルトは512です  
  --num_source（int）：Attentionのソースイメージの数。デフォルトは2です。Largeにはより多くのGPUメモリが必要です  
  --assets_dir（str）：アセットディレクトリ。configurationとすべての事前トレーニング済みチェックポイントが必要  
  --output_dir（str）：出力ディレクトリ  
  --src_path（str）：ソース入力情報  

[使用画像](https://livedoor.blogimg.jp/chargomannsei/imgs/6/6/66e10295-s.jpg)

In [ ]:
%cd /content/iPERCore
!rm -rf images
!mkdir images


#@markdown  front_only:正面画像のみ、front_back:正面、背面画像、fron_back_bg:正面、背面、背景画像
model_type = "front_only"  #@param ["front_only", "front_back", "front_back_bg"] {allow-input: false}

gpu_ids = "0"
#@markdown 出力画像サイズ
image_size = 512 #@param {type:"integer"}
num_source = 2
assets_dir = "/content/iPERCore/assets"
output_dir = "/content/iPERCore/results"
cfg_path = "/content/iPERCore/assets/configs/deploy.toml"

# model設定
model_id = "donald_trump_2"
if model_type == "front_back":
  model_id = "axing_1"
elif model_type == "front_back_bg":
  model_id = "afan_6=ns=2"

%cd /content/iPERCore/images
# src画像アップロード
if model_type == "front_only":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())

  front_img_full_path = os.path.join("/content/iPERCore/images", front_img[0])
  src_path = "\"path?=" + front_img_full_path + ",name?=front_only\""
elif model_type == "front_back":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())
  print("背面画像をアップロードしてください")
  back_img = files.upload()
  back_img = list(back_img.keys())

  src_path = "\"path?=/content/iPERCore/images,name?=front_back\""
elif model_type == "front_back_bg":
  print("正面画像をアップロードしてください")
  front_img = files.upload()
  front_img = list(front_img.keys())
  print("背面画像をアップロードしてください")
  back_img = files.upload()
  back_img = list(back_img.keys())
  print("背景画像をアップロードしてください")
  bg_img = files.upload()
  bg_img = list(bg_img.keys())

  bg_img_full_path = os.path.join("/content/iPERCore/images", bg_img[0])
  src_path = "\"path?=/content/iPERCore/images," \
             "name?=front_back_bg," \
             "bg_path?=" + bg_img_full_path + "\""



In [ ]:
%cd /content/iPERCore

!rm -rf videos
!mkdir videos

video_url = 'https://www.youtube.com/watch?v=VSSPwzSflr0' #@param {type:"string"}

#@markdown 動画の切り抜き範囲(秒)を指定してください。\
#@markdown 30秒以上の場合OOM発生の可能性が高いため注意
start_sec =  3#@param {type:"integer"}
end_sec =  10#@param {type:"integer"}

(start_pt, end_pt) = (start_sec, end_sec)

download_resolution = 360
full_video_path = '/content/iPERCore/videos/full_video.mp4'
subclip_path = '/content/iPERCore/videos/subclip.mp4'
video_name = 'subclip'

# 動画ダウンロード
ydl_opts = {'format': f'best[height<={download_resolution}]', 'overwrites': True, 'outtmpl': full_video_path}
with YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])

# 指定区間切り抜き
with VideoFileClip(full_video_path) as video:
    subclip = video.subclip(start_pt, end_pt)
    subclip.write_videofile(subclip_path)

In [ ]:
ref_path = "\"path?=" + subclip_path + "," \
             "name?=" + video_name + "," \
             "pose_fc?=400\""

In [ ]:
%%time
!rm -rf {output_dir}

%cd /content/iPERCore/

!python -m iPERCore.services.run_imitator  \
  --gpu_ids     $gpu_ids       \
  --num_source  $num_source    \
  --image_size  $image_size    \
  --output_dir  $output_dir    \
  --model_id    $model_id      \
  --cfg_path    $cfg_path      \
  --src_path    $src_path      \
  --ref_path    $ref_path

In [ ]:
%cd /content/iPERCore/
results_video = "/content/iPERCore/results/primitives/" + model_type + "/synthesis/imitations/" + model_type + "-" + video_name + ".mp4"

clip = VideoFileClip(results_video)
clip = resize(clip, height=420)
clip.ipython_display()